# Character Impact Project (Part 2) - Cleaning Data from the IMDb Dataset
## Data Wrangling
### Alden Chico

---

## Premise

For the first step of our Character Impact Project, we need to wrangle the data from the IMDb Datasets and spit out a table with all the relevant information they have about <i>The Office</i>. IMDb grants the general public access to subsets of their data for non-commercial and personal use on their website 
<a href="https://www.imdb.com/interfaces/"> here </a>
. From this website, there are five tables of interest to us:
1. ```title.basics.tsv```: Contains basic information for <i>The Office</i> like its opening and closing years.
2. ```title.episode.tsv```: Contains  episode information for <i>The Office</i> like the season and episode number for every particular episode.
3. ```title.ratings.tsv```: Contains user rating information associated to each episode.
4. ```title.principals.tsv```: Contains top-billed talent for each episode of <i>The Office</i> like directors, writers, and actors/actresses.
5. ```name.basics.tsv```: Contains information about every actor/actress in the IMDb dataset.

<figure>
<img src="https://i.etsystatic.com/11276721/d/il/7ccdef/1190816607/il_340x270.1190816607_idbd.jpg?version=0" alt="Dwight Schrute: “Why are all these people here? There are too many people on this Earth. We need a new plague.”" class="center">
<figcaption><center><b>Dwight Schrute: “Why are all these people here? There are too many people on this earth. We need a new plague.”</b></center></figcaption>
</figure> 


There is one unifying variable that ties all these tables together: ```tconst```. ```tconst``` (or ```nconst``` for the ```name.basics table```) is a ***primary key*** associated with every television episode, series, and actor/actress in the IMDb dataset. Relating ```tconst``` from one table to another in an appropriate fashion will allow us to filter out the millions of rows of information in the IMDb dataset to the relevant information we want from <i>The Office</i>. This concept and more will be explained as we go through our data wrangling steps :)

---

## Step 1: Setup

In [1]:
import pandas as pd
import glob
import requests
import itertools
from bs4 import BeautifulSoup

To begin our data wrangling process, we must set up the appropriate tools for the job. If you downloaded my repository for this project from GitHub, you can run ```pip install -r reference/2-Data_Cleaning/requirements.txt``` from the root repository to download the packages you need for this project.

In [2]:
'''Create a list of file names from the directory that stores the IMDb Databases dataset'''

file_list = glob.glob('reference/2-Data_Cleaning/IMDb-Datasets/*.tsv')
file_list

['reference/2-Data_Cleaning/IMDb-Datasets/title.ratings.tsv',
 'reference/2-Data_Cleaning/IMDb-Datasets/title.principals.tsv',
 'reference/2-Data_Cleaning/IMDb-Datasets/name.basics.tsv',
 'reference/2-Data_Cleaning/IMDb-Datasets/title.basics.tsv',
 'reference/2-Data_Cleaning/IMDb-Datasets/title.episode.tsv']

Next, we have to download all the relevant IMDb datasets and reference them appropriately so our program can find them. Here, all the IMDb datasets are held in the ```reference/2-Data-Cleaning/IMDb-Datasets``` folder of our project. All these files are loaded onto GitHub, so downloading the project will grant access to these files off the bat. 

To reference these files in our program, we will use the Python ```glob``` library. The glob function pulls all the file names with their paths from the project folder as a list of strings for this program.

In [3]:
''' Create a dictionary of DataFrames from the tsv file's data''' 

imdb_dfs = {}
for file in file_list:
    df_name = file.split('/')[-1]
    df_name = df_name.split('.')
    df_name = df_name[0] + '.' + df_name[1] + '_df'
    imdb_dfs[df_name] = pd.read_csv(file, sep='\t', low_memory = False)

Now that we have a list of file names loaded onto the ```file_list``` variable, we have to  take the information from every file in the list and store them as pandas DataFrames. Simply put, pandas DataFrames are tables where each row is associated to an observation and each column is associated to a variable. Every table for this project has their own definitions for observations and variables which will be explained as they're observed.

Now that all the files are loaded onto pandas DataFrames, we are now ready to clean up the data and pull all the information for <i>The Office</i> from IMDb's dataset!

<figure>
<img src="https://img2.thejournal.ie/inline/3576809/original/?width=630&version=3576809" alt="Jim, what a guy...”" class="center"style="width:533px;height:300px;">
<figcaption><center><b>That Wasn't Too Bad :D</b></center></figcaption>
</figure> 


---

## Step 2: Gathering Basic Information for <i>The Office</i> from ```title.basics_df```

In [4]:
imdb_dfs['title.basics_df'].head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


A quick observation of ```title.basics_df``` shows that the table contains ```primaryTitle``` information that denotes the title for every movie, short, TV series, TV episode, and video in IMDb's dataset. Every row is an individual piece of media in the IMDb dataset. What we are concerned about now is finding information for the US version of <i>The Office</i>. Well, since we know the name of the show and its ```titleType``` (TV series), that would be a good place to start our filtering. To keep track of our filtering, we will be storing information for <i>The Office</i> in its own DataFrame called ```the_office_df```.

In [5]:
'''Filter out everything from the title.basics_df not related to TV Series called The Office'''

the_office_df = imdb_dfs['title.basics_df'].loc[(imdb_dfs['title.basics_df'].primaryTitle == 'The Office') 
                                              & (imdb_dfs['title.basics_df'].titleType == 'tvSeries') , :]
the_office_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
109696,tt0112108,tvSeries,The Office,The Office,0,1995,\N,30,Comedy
279011,tt0290978,tvSeries,The Office,The Office,0,2001,2003,30,"Comedy,Drama"
371064,tt0386676,tvSeries,The Office,The Office,0,2005,2013,22,Comedy
1977861,tt1791001,tvSeries,The Office,Ha-Misrad,0,2010,\N,25,Comedy
2354182,tt2186395,tvSeries,The Office,The Office,0,2012,\N,5,Comedy


Now that we've filtered the ```title.basics_df``` DataFrame for all TV series called <i>The Office</i>, we now realize that there are 5 series in the IMDb dataset that all have <i>The Office</i> moniker. Well, a quick Google search shows that <i>The Office (USA)</i> started on 2005 and ended on 2013. We should filter out the ```startYear``` column to get the information we need for our version of <i>The Office</i>.

In [6]:
''' Filter out the_office_df for The Office TV Series that started in 2005 '''

the_office_df = the_office_df.loc[the_office_df.startYear == '2005', :]
the_office_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
371064,tt0386676,tvSeries,The Office,The Office,0,2005,2013,22,Comedy


So now we have a row associated to the US version of <i>The Office</i>. So now what? Well, as I mentioned in the premise, we want ```tconst``` because that will help us link this table to the rest of the tables in the IMDb dataset. All the other information in this row isn't relevant to us anymore.

In [7]:
''' Gather a list of columns called the_office_columns that will hold all the relevant column information for
    the_office_df '''

the_office_columns = ['primaryTitle', 'tconst']
the_office_df = the_office_df[the_office_columns]
the_office_df

,primaryTitle,tconst
371064,The Office,tt0386676


<figure>
<img src="https://www.yourtango.com/sites/default/files/styles/body_image_default/public/2018/theoffice11.jpg" alt="Jim, what a guy...”" class="center"style="width:506px;height:300px;">
<figcaption><center><b>Filtering Out Datasets is a Piece of Cake :3</b></center></figcaption>
</figure> 


---

## Step 3: Gathering Information for Season 1 of <i>The Office</i> from ```title.episode_df```

In [8]:
imdb_dfs['title.episode_df'].head()

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1,9
1,tt0042816,tt0989125,1,17
2,tt0042889,tt0989125,\N,\N
3,tt0043426,tt0040051,3,42
4,tt0043631,tt0989125,2,16


For the ```title.episode_df```, every row is associated to a unique ```tconst``` associated to TV episodes listed on IMDb's dataset. On top of that, we also have this column called ```parentTconst``` that can be associated to the TV Series for which that episode belongs to. Well, from our last step, we retrieved the ```tconst``` associated to <i>The Office (USA)</i>. All we have to do now is match ```parentTconst``` from ```title.episode_df``` to ```tconst``` in ```the_office_df``` to retrieve all the episode information for the series.

In [9]:
''' Merge the_office_df with title.episode_df on tconst and parentTconst, respectively '''

the_office_df = the_office_df.merge(imdb_dfs['title.episode_df'], left_on=the_office_df.tconst,
                                    right_on=imdb_dfs['title.episode_df'].parentTconst,
                                    suffixes=('_series', '_episode'), how='inner')
the_office_df.head()

,key_0,primaryTitle,tconst_series,tconst_episode,parentTconst,seasonNumber,episodeNumber
0,tt0386676,The Office,tt0386676,tt0664510,tt0386676,1,5
1,tt0386676,The Office,tt0386676,tt0664511,tt0386676,2,11
2,tt0386676,The Office,tt0386676,tt0664512,tt0386676,2,15
3,tt0386676,The Office,tt0386676,tt0664513,tt0386676,2,10
4,tt0386676,The Office,tt0386676,tt0664514,tt0386676,1,2


After joining the two tables in this manner, we see that we now have a table that contains information for ALL the episodes and seasons in <i>The Office</i>. We're only concerned about Season 1 of our show, so we can filter out all the information from our table associated with the other seasons.

In [10]:
'''Filter out all episode information not relevant to season 1'''

the_office_df = the_office_df.loc[the_office_df.seasonNumber == '1', :].reset_index(drop=True)
the_office_df

,key_0,primaryTitle,tconst_series,tconst_episode,parentTconst,seasonNumber,episodeNumber
0,tt0386676,The Office,tt0386676,tt0664510,tt0386676,1,5
1,tt0386676,The Office,tt0386676,tt0664514,tt0386676,1,2
2,tt0386676,The Office,tt0386676,tt0664517,tt0386676,1,3
3,tt0386676,The Office,tt0386676,tt0664518,tt0386676,1,6
4,tt0386676,The Office,tt0386676,tt0664521,tt0386676,1,1
5,tt0386676,The Office,tt0386676,tt0664523,tt0386676,1,4


Our table only contains information for the first season of <i>The Office</i> now. If you noticed, we have a column called ```key_0``` that was created when we merged the ```title.episode_df``` table to our ```the_office_df``` table. ```key_0``` just contains the same information as ```tconst_series```. It makes sense to remove this redundant column before moving onto our next step: associating each episode with their respective ratings. While we're at it, we'll also join the episode titles to ```the_office_df``` using the same process that we used in the last step.


<figure>
<img src="https://i.pinimg.com/originals/40/e3/d2/40e3d2fcbd37b5b5839aa68a3c69ba9c.jpg" alt="Dwight wearing the mask from THAT CPR scene xD" class="center"style="width:463px;height:300px;">
<figcaption><center><b>Eliminate. Unnecessary. Columns...</b></center></figcaption>
</figure> 



In [11]:
''' Reformat the_office_columns now that there's a tconst for the series and a tconst for the episode '''

the_office_columns[1] = 'tconst_series'
new_columns = ['tconst_episode', 'seasonNumber', 'episodeNumber']
for column in new_columns:
    the_office_columns.append(column)
the_office_df = the_office_df[the_office_columns].sort_values(by=['episodeNumber']).reset_index(drop=True)
the_office_df

,primaryTitle,tconst_series,tconst_episode,seasonNumber,episodeNumber
0,The Office,tt0386676,tt0664521,1,1
1,The Office,tt0386676,tt0664514,1,2
2,The Office,tt0386676,tt0664517,1,3
3,The Office,tt0386676,tt0664523,1,4
4,The Office,tt0386676,tt0664510,1,5
5,The Office,tt0386676,tt0664518,1,6


In [12]:
''' Add the name of the each episode to their respective rows using the title.basics_df table'''

the_office_df = the_office_df.merge(imdb_dfs['title.basics_df'], left_on=the_office_df.tconst_episode,
                                         right_on=imdb_dfs['title.basics_df'].tconst, how='inner',
                                         suffixes=['_series','_episode'])

the_office_columns= ['tconst_series', 'tconst_episode', 'primaryTitle_series', 'primaryTitle_episode', 
                           'seasonNumber', 'episodeNumber']
the_office_df = the_office_df[the_office_columns].sort_values(by=['episodeNumber'])
the_office_df

,tconst_series,tconst_episode,primaryTitle_series,primaryTitle_episode,seasonNumber,episodeNumber
0,tt0386676,tt0664521,The Office,Pilot,1,1
1,tt0386676,tt0664514,The Office,Diversity Day,1,2
2,tt0386676,tt0664517,The Office,Health Care,1,3
3,tt0386676,tt0664523,The Office,The Alliance,1,4
4,tt0386676,tt0664510,The Office,Basketball,1,5
5,tt0386676,tt0664518,The Office,Hot Girl,1,6


---

## Step 4: Associating Each Episode with Their Respective Ratings Using ```title.ratings_df```

In [13]:
imdb_dfs['title.ratings_df'].head()

,tconst,averageRating,numVotes
0,tt0000001,5.8,1513
1,tt0000002,6.3,184
2,tt0000003,6.6,1160
3,tt0000004,6.3,113
4,tt0000005,6.2,1865


By now, you may have noticed a pattern to our process. Every row in these tables has a ```tconst``` that we can use to associate to rows in our table for <i>The Office</i>. ```title.ratings_df``` is no exception. We will associate every episode from Season 1 of <i>The Office</i> with the ```title.ratings_df``` through ```tconst_episode``` and ```tconst```, respectively. Here are the results.

In [14]:
''' Merge the_office_df with title.ratings_df on tconst_episode and tconst, respectively '''

the_office_df = the_office_df = the_office_df.merge(imdb_dfs['title.ratings_df'], left_on=the_office_df.tconst_episode,
                                    right_on=imdb_dfs['title.ratings_df'].tconst,
                                    suffixes=('_episode', '_ratings'), how='inner')
the_office_df

,key_0,tconst_series,tconst_episode,primaryTitle_series,primaryTitle_episode,seasonNumber,episodeNumber,tconst,averageRating,numVotes
0,tt0664521,tt0386676,tt0664521,The Office,Pilot,1,1,tt0664521,7.6,3644
1,tt0664514,tt0386676,tt0664514,The Office,Diversity Day,1,2,tt0664514,8.3,3507
2,tt0664517,tt0386676,tt0664517,The Office,Health Care,1,3,tt0664517,7.9,2939
3,tt0664523,tt0386676,tt0664523,The Office,The Alliance,1,4,tt0664523,8.1,2846
4,tt0664510,tt0386676,tt0664510,The Office,Basketball,1,5,tt0664510,8.4,3132
5,tt0664518,tt0386676,tt0664518,The Office,Hot Girl,1,6,tt0664518,7.8,2820


Well, after an easy merge between the two tables, we now see that every episode from the show is associated with their respective ratings. The next thing we should do is clean up the redundant columns from the table and move onto our next step: associating each episode with their respective principal cast.

In [15]:
''' Clean the_office_df so that it contains only relevant column information '''

the_office_columns.append('averageRating')
the_office_df = the_office_df[the_office_columns]
the_office_df

,tconst_series,tconst_episode,primaryTitle_series,primaryTitle_episode,seasonNumber,episodeNumber,averageRating
0,tt0386676,tt0664521,The Office,Pilot,1,1,7.6
1,tt0386676,tt0664514,The Office,Diversity Day,1,2,8.3
2,tt0386676,tt0664517,The Office,Health Care,1,3,7.9
3,tt0386676,tt0664523,The Office,The Alliance,1,4,8.1
4,tt0386676,tt0664510,The Office,Basketball,1,5,8.4
5,tt0386676,tt0664518,The Office,Hot Girl,1,6,7.8


<figure>
<img src="https://m.media-amazon.com/images/M/MV5BMTY0MjUzNTcwMl5BMl5BanBnXkFtZTgwODEzMjk4MzI@._V1_SY1000_CR0,0,666,1000_AL_.jpg" alt="Michael wants to leeeave :(" class="center"style="width:233px;height:300px;">
<figcaption><center><b>Yes Michael, We're Almost Done Here</b></center></figcaption>
</figure> 


---

## Step 5: Finding the Principal Talent for Each Episode of <i>The Office</i> Using ```title.principals_df```

In [16]:
imdb_dfs['title.principals_df'].head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Herself""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N


Alright, here we go. The final step. We'll do the same thing we've done 3 times prior and use ```tconst``` from ```title.principals_df``` with our table for <i>The Office</i> and remove all the redundant columns. Here, we're only interested in gathering information about the characters from the show since those are the only people present on screen during the episode. After we're done with this step, we should be good to go on some preliminary analysis on this dataset. **Let's do this!!!**

In [17]:
''' Merge the_office_df with title.principals_df on tconst_episode and tconst, respectively '''

the_office_df = the_office_df = the_office_df.merge(imdb_dfs['title.principals_df'], 
                                                    left_on=the_office_df.tconst_episode,
                                                    right_on=imdb_dfs['title.principals_df'].tconst,
                                                    suffixes=('_episode', '_principals'), how='inner')
the_office_df.head()

,key_0,tconst_series,tconst_episode,primaryTitle_series,primaryTitle_episode,seasonNumber,episodeNumber,averageRating,tconst,ordering,nconst,category,job,characters
0,tt0664521,tt0386676,tt0664521,The Office,Pilot,1,1,7.6,tt0664521,10,nm0810545,cinematographer,director of photography,\N
1,tt0664521,tt0386676,tt0664521,The Office,Pilot,1,1,7.6,tt0664521,1,nm0136797,actor,\N,"[""Michael Scott""]"
2,tt0664521,tt0386676,tt0664521,The Office,Pilot,1,1,7.6,tt0664521,2,nm0933988,actor,\N,"[""Dwight Schrute""]"
3,tt0664521,tt0386676,tt0664521,The Office,Pilot,1,1,7.6,tt0664521,3,nm1024677,actor,\N,"[""Jim Halpert""]"
4,tt0664521,tt0386676,tt0664521,The Office,Pilot,1,1,7.6,tt0664521,4,nm0278979,actress,\N,"[""Pam Beesly""]"


In [18]:
''' Reorganize the_office_df by ordering on the principal list'''

the_office_df = the_office_df.sort_values(by=['episodeNumber', 'ordering'])
the_office_df.head()

,key_0,tconst_series,tconst_episode,primaryTitle_series,primaryTitle_episode,seasonNumber,episodeNumber,averageRating,tconst,ordering,nconst,category,job,characters
1,tt0664521,tt0386676,tt0664521,The Office,Pilot,1,1,7.6,tt0664521,1,nm0136797,actor,\N,"[""Michael Scott""]"
2,tt0664521,tt0386676,tt0664521,The Office,Pilot,1,1,7.6,tt0664521,2,nm0933988,actor,\N,"[""Dwight Schrute""]"
3,tt0664521,tt0386676,tt0664521,The Office,Pilot,1,1,7.6,tt0664521,3,nm1024677,actor,\N,"[""Jim Halpert""]"
4,tt0664521,tt0386676,tt0664521,The Office,Pilot,1,1,7.6,tt0664521,4,nm0278979,actress,\N,"[""Pam Beesly""]"
5,tt0664521,tt0386676,tt0664521,The Office,Pilot,1,1,7.6,tt0664521,5,nm0477129,director,\N,\N


In [19]:
''' Clean the_office_df so it only shows relevant column information '''

new_columns = ['nconst', 'category', 'characters']
for column in new_columns:
    the_office_columns.append(column)

In [20]:
''' Only include information related to people who are associated as actor '''

the_office_df = the_office_df.loc[the_office_df.category == 'actor', 
                                  the_office_columns].reset_index(drop=True)
the_office_df.head()

,tconst_series,tconst_episode,primaryTitle_series,primaryTitle_episode,seasonNumber,episodeNumber,averageRating,nconst,category,characters
0,tt0386676,tt0664521,The Office,Pilot,1,1,7.6,nm0136797,actor,"[""Michael Scott""]"
1,tt0386676,tt0664521,The Office,Pilot,1,1,7.6,nm0933988,actor,"[""Dwight Schrute""]"
2,tt0386676,tt0664521,The Office,Pilot,1,1,7.6,nm1024677,actor,"[""Jim Halpert""]"
3,tt0386676,tt0664514,The Office,Diversity Day,1,2,8.3,nm0136797,actor,"[""Michael Scott""]"
4,tt0386676,tt0664514,The Office,Diversity Day,1,2,8.3,nm0933988,actor,"[""Dwight Schrute""]"


Wait a minute... There's something wrong here. Every episode only has three actors involved. There are definitely more than three people on screen per episode. It looks like we have to do something else to gather the information for the cast and characters for each episode. The best thing we can do is go on the IMDb website page for an episode of <i>The Office</i> and see if we can find more information about the characters involved in that episode. For now, let's just get rid of the columns that we added from this step of the process.

In [21]:
''' Clean the_office_df from all irrelevant columns '''

for column in new_columns:
    del the_office_df[column]
    the_office_columns.remove(column)
the_office_df = the_office_df.drop_duplicates().reset_index(drop=True)
the_office_df

,tconst_series,tconst_episode,primaryTitle_series,primaryTitle_episode,seasonNumber,episodeNumber,averageRating
0,tt0386676,tt0664521,The Office,Pilot,1,1,7.6
1,tt0386676,tt0664514,The Office,Diversity Day,1,2,8.3
2,tt0386676,tt0664517,The Office,Health Care,1,3,7.9
3,tt0386676,tt0664523,The Office,The Alliance,1,4,8.1
4,tt0386676,tt0664510,The Office,Basketball,1,5,8.4
5,tt0386676,tt0664518,The Office,Hot Girl,1,6,7.8


<figure>
<img src="https://m.media-amazon.com/images/M/MV5BMjUwNjI1MjIxOV5BMl5BanBnXkFtZTgwMTI1NTU1MjE@._V1_.jpg" alt="Don't cry Jan :(" class="center"style="width:400px;height:300px;">
<figcaption><center><b>It's Okay Jan, You're Not Getting Fired (yet...)</b></center></figcaption>
</figure> 


---

## Step 6: Gathering Data the Dirty Way! (***that's what she said***)

Well, here's what the IMDb page for the first episode looks like. The URL for this webpage is **https://www.imdb.com/title/tt0664521/?ref_=ttep_ep1**. If you look closer at this URL link, you'll see that tt0664521 is the ```tconst``` for that episode, and the 1 from the end of the link means that it's the first episode of the season. To confirm this, here's the URL link for the second episode's IMDb page: **https://www.imdb.com/title/tt0664514/?ref_=ttep_ep2**.

<figure>
<img src="reference/2-Data_Cleaning/IMDb-Screenshot.png" alt="IMDb Screenshot" class="center"style="width:523px;height:400px;">
<figcaption><center><b>Yes, I'm aware of the number of tabs on my browser right now.</b></center></figcaption>
</figure> 



And if you notice, there's a cast list on this page with all the relevant cast members for that episode which is exactly what we need. If you dive into the HTML code for this page, you can find that all this information is kept inside ```<div class="article" id="titleCast"> ... </div>```.

<figure>
<img src="reference/2-Data_Cleaning/IMDB-Inspect.png" alt="IMDb Inspect" class="center"style="width:600px;height:400px;">
<figcaption><center><b>Look at the code on the right, highlighted in blue.</b></center></figcaption>
</figure> 


So in order to get the cast information for each episode of <i>The Office</i> we have to extract the HTML code from the web page associated with an episode, gather a list of the cast and character information from that episode, and repeat for all six episodes of the season. **Easy.**

<figure>
<img src="https://m.media-amazon.com/images/M/MV5BMTg1MDc1NjI2OF5BMl5BanBnXkFtZTcwOTA4NTIzNQ@@._V1_.jpg" alt="Toby's the Scranton Strangler XC" class="center"style="width:400px;height:300px;">
<figcaption><center><b>Ew. Toby.</b></center></figcaption>
</figure> 




In [22]:
the_office_df = the_office_df.astype('object')
for ep_num, ep in the_office_df.tconst_episode.iteritems():
    
    #Construct the IMDb URL and extract HTML data
    x = ep_num + 1
    url = f'https://www.imdb.com/title/{ep}/?ref_=ttep_ep{x}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    
    #Construct the first list of cast members from HTML data
    cast_odd = soup.find_all('tr', class_='odd')
    cast_odd_list = []
    for cast_member in cast_odd:
        cast_odd_list.append(str(cast_member.find_all('a')[0].contents[0])
                             .split('title=')[1].split('\"')[1].replace('\n','').strip())

    #Construct the second list of cast members from HTML data
    cast_even = soup.find_all('tr', class_='even')
    cast_even_list = []
    for cast_member in cast_even:
        cast_even_list.append(str(cast_member.find_all('a')[0].contents[0])
                             .split('title=')[1].split('\"')[1].replace('\n','').strip())

    #Construct a merged list between the two cast members lists
    cast_list = list(itertools.chain
                           .from_iterable(zip(cast_odd_list, cast_even_list)))

    #Construct the first list of characters from HTML data
    characters_odd = soup.find_all('tr', class_='odd')
    characters_odd_list = []
    for character in characters_odd:
        characters_odd_list.append(character.find('td', class_='character')
                                   .text.replace('\n','').strip())
    #Construct the second list of characters from HTML data
    characters_even = soup.find_all('tr', class_='even')
    characters_even_list = []
    for character in characters_even:
        characters_even_list.append(character.find('td', class_='character')
                                    .text.replace('\n','').split('(')[0].strip())

    #Construct a merged list from the two characters lists
    characters_list = list(itertools.chain
                               .from_iterable(zip(characters_odd_list, characters_even_list)))
    
    #Associate the cast and characters lists to the_office_df
    the_office_df.loc[ep_num, 'cast'] = 'a'
    the_office_df.loc[ep_num, 'characters'] = 'a'
    the_office_df.loc[ep_num, 'cast'] = cast_list
    the_office_df.loc[ep_num, 'characters'] = characters_list
    
the_office_df

,tconst_series,tconst_episode,primaryTitle_series,primaryTitle_episode,seasonNumber,episodeNumber,averageRating,cast,characters
0,tt0386676,tt0664521,The Office,Pilot,1,1,7.6,"[Steve Carell, Rainn Wilson, John Krasinski, J...","[Michael Scott, Dwight Schrute, Jim Halpert, P..."
1,tt0386676,tt0664514,The Office,Diversity Day,1,2,8.3,"[Steve Carell, Rainn Wilson, John Krasinski, J...","[Michael Scott, Dwight Schrute, Jim Halpert, P..."
2,tt0386676,tt0664517,The Office,Health Care,1,3,7.9,"[Steve Carell, Rainn Wilson, John Krasinski, J...","[Michael Scott, Dwight Schrute, Jim Halpert, P..."
3,tt0386676,tt0664523,The Office,The Alliance,1,4,8.1,"[Steve Carell, Rainn Wilson, John Krasinski, J...","[Michael Scott, Dwight Schrute, Jim Halpert, P..."
4,tt0386676,tt0664510,The Office,Basketball,1,5,8.4,"[Steve Carell, Rainn Wilson, John Krasinski, J...","[Michael Scott, Dwight Schrute, Jim Halpert, P..."
5,tt0386676,tt0664518,The Office,Hot Girl,1,6,7.8,"[Steve Carell, Rainn Wilson, John Krasinski, J...","[Michael Scott, Dwight Schrute, Jim Halpert, P..."


This is the largest chunk of code yet, so I'll break it down for you. First, we enter a loop that will go through all the ```tconst_episode``` values in our table for <i>The Office</i> one by one. The first thing we do inside the loop is construct a URL link for that episode's IMDb page and extract the HTML data from the link using a get function defined in the python ```requests``` library. The ```requests``` library simply allows us to connect to the Internet and fetch information from the IMDb webpage for our program.

Now that we have the HTML text data, we can use the ```BeautifulSoup``` library to parse through the HTML data for cast and character information from the webpage. ```BeautifulSoup``` uses HTML conventions to make parsing through HTML data easier, but doing so still takes a lot of trial and error. 

After parsing through the relevant HTML data on our webpage, we extract two separate lists for cast members and two separate lists for characters. This is because the HTML file separates the rows in  ```titleCast``` into even and odd classes. In order to rearrange the cast into one list and the characters into one list, we use the ```itertools``` library that contains the ```.chain``` function that can join the cast lists and characters lists in their appropriate order.

From there, all we have to do now is associate the cast and characters lists to the appropriate episode and repeate for every episode.

<figure>
<img src="https://m.media-amazon.com/images/M/MV5BMTM2OTg0MDE0OF5BMl5BanBnXkFtZTcwMTA5Nzk0NA@@._V1_.jpg" alt="Can we go home yet?" class="center"style="width:200px;height:300px;">
<figcaption><center><b>Can we please go home Alden?</b></center></figcaption>
</figure> 



Yes Michael, we can go home after we check our work. Here's a list of characters for the Season 1 Episode 2 of <i>The Office</i>.

In [23]:
the_office_df['characters'][1]

['Michael Scott',
 'Dwight Schrute',
 'Jim Halpert',
 'Pam Beesly',
 'Ryan Howard',
 'Mr. Brown',
 'Stanley Hudson',
 'Kevin Malone',
 'Meredith Palmer',
 'Kelly Kapoor',
 'Toby Flenderson',
 'Angela Martin',
 'Oscar Martinez',
 'Phyllis Lapin']

And if we do a quick check on the episode's IMDb page, it looks like we got the characters in the right order on the cast list!

<figure>
<img src="reference/2-Data_Cleaning/IMDb-Check.png" alt="IMDb Check" class="center"style="width:572px;height:400px;">
<figcaption><center><b>Compare the Cast Members on the List with the List Above</b></center></figcaption>
</figure> 

And we're done! All we have to do now is save the_office_df to a CSV file in our project output folder and move onto the next step of the project: Gathering insights from the data that we've collected so far :) Until next time!

In [24]:
the_office_df.to_csv("output/the_office.csv")